In [ ]:
 # This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'backup'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# Change dariectory to current folder
%cd /content/drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/backup


In [ ]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [ ]:
import sentencepiece as spm

de_vocab_file = 'de.model'
en_vocab_file = 'en.model'

de_vocab = spm.SentencePieceProcessor()
en_vocab = spm.SentencePieceProcessor()

# de, en vocab 로드
de_vocab.load(de_vocab_file)
en_vocab.load(en_vocab_file)

True

In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv')

In [ ]:
# data.py

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler

# mt Dataset
class MtDataset(Dataset):
  def __init__(self, src_vocab, trg_vocab, df, src_name, trg_name):
    self.src_vocab  = src_vocab
    self.trg_vocab = trg_vocab
    self.src_train = []
    self.trg_train = []

    for idx, row in df.iterrows():
      src_line = row[src_name]
      trg_line = row[trg_name]
      if type(src_line) != str or type(trg_line) != str:
        continue
      # src 문장, trg 문장 각각 tokenize
      self.src_train.append(src_vocab.encode_as_ids(src_line))
      self.trg_train.append(trg_vocab.encode_as_ids(trg_line))

  def __len__(self):
    assert len(self.src_train) == len(self.trg_train)
    return len(self.src_train)

  def __getitem__(self, idx):
    return (torch.tensor(self.src_train[idx]), torch.tensor(self.trg_train[idx]))

'''
read the csv in __init__ but leave the reading of images to __getitem__
'''


# mt data collate_fn
# 배치 단위로 데이터 처리
def mt_collate_fn(inputs):
  enc_inputs, dec_inputs = list(zip(*inputs)) # to do

  # 입력 길이가 다르므로 입력 최대 길이에 맟춰 padding(0) 추가
  enc_inputs = torch.nn.utils.rnn.pad_sequence(enc_inputs, batch_first=True)
  dec_inputs = torch.nn.utils.rnn.pad_sequence(dec_inputs, batch_first=True)

  batch = [
      enc_inputs,
      dec_inputs
  ]

  return batch # DataLoader iterate 할 때 return됨


# DataLoader
def build_mt_data_loader(src_vocab, trg_vocab, df, src_name, trg_name, args, shuffle=True):
  # Dataset 생성
  dataset = MtDataset(src_vocab, trg_vocab, df, src_name, trg_name)
  if 1 < args['n_gpu'] and shuffle:
    sampler = DistributedSampler(dataset)
    loader = DataLoader(dataset, batch_size=args['batch'], sampler=sampler, collate_fn=mt_collate_fn)
  else:
    sampler = None
    loader = DataLoader(dataset, batch_size=args['batch'], sampler=sampler, shuffle=shuffle, collate_fn=mt_collate_fn)

  return loader, sampler

In [ ]:
dataset = MtDataset(en_vocab, de_vocab, train_df, 'en', 'de')

In [ ]:
dataset.__len__()

206112

In [ ]:
dataset.__getitem__(123456)

(tensor([  35,  891,   41,    8,  436,  653,  210,   54,   25,  690,   41,  225,
           50,  311, 7956, 7933, 1375, 1977,   25,  221, 6576, 7951,  109,   99,
         4232, 7951,   36,   60, 7956, 7938,  117,  225,  109,  711, 1023, 7953]),
 tensor([ 145,  850, 7937,   18, 4993, 1133,   61, 7937,   46,  922, 7937,  107,
          207,   59,  491, 6109, 4482,  232,  452,  258, 4958,  137, 3889, 7937,
           91, 7541, 2196,   35,   83,   61, 7937,  303,  574,   91,  550, 3398,
         2141, 7940]))

In [ ]:
dataset[123456]

(tensor([  35,  891,   41,    8,  436,  653,  210,   54,   25,  690,   41,  225,
           50,  311, 7956, 7933, 1375, 1977,   25,  221, 6576, 7951,  109,   99,
         4232, 7951,   36,   60, 7956, 7938,  117,  225,  109,  711, 1023, 7953]),
 tensor([ 145,  850, 7937,   18, 4993, 1133,   61, 7937,   46,  922, 7937,  107,
          207,   59,  491, 6109, 4482,  232,  452,  258, 4958,  137, 3889, 7937,
           91, 7541, 2196,   35,   83,   61, 7937,  303,  574,   91,  550, 3398,
         2141, 7940]))

In [ ]:
args = {
    'n_gpu': 1,
    'batch': 2 #256
}

loader, sampler = build_mt_data_loader(en_vocab, de_vocab, train_df, 'en', 'de', args)

cnt = 0

for d in loader:
  if (cnt < 1):
    print(d)
    cnt+= 1
  else:
    break

[tensor([[1777,  566, 4475, 4676, 6065, 7953,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0],
        [ 353,  787, 7982, 3101,    8,  955,  318,  124,  480,   25, 1337,  681,
         4283,   25, 2594,  903, 7953]]), tensor([[ 116,   18,  480,   35, 1181,   49,  982,   66, 4722, 7940,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [3718,  377,  227,  833, 5867,  269,  138,   18, 2383,  116, 1445, 2888,
         5004, 6342, 7940,  342, 5929,  232, 7940]])]
